In [ ]:

#rasha code
#clean the data from na values & pick the columns
scaled_df = sp500.copy()
scaled_df = drop_nan(scaled_df)
scaled_df = scaled_df.loc[:, ['WMA_signal', 'macd_signal', 'RSI', '%K_fast', '%D_fast', '%D_slow', 'Bollinger Diff', 'WPR', 'OBV', 'log_return']]

In [ ]:
groups = 10
train = np.array(scaled_df)
X, y = train[:, :-1], train[:, -1]

def build_rolling_calendar(start_date, end_date, out_of_sample_size, runs, n_gap):
    days = (end_date - start_date).days

    in_sample_size = (100 - out_of_sample_size) / 100
    out_of_sample_size = out_of_sample_size / 100

    total_days_per_run = round(days / (runs * out_of_sample_size + in_sample_size))
    in_sample_days_per_run = round(total_days_per_run * in_sample_size)
    out_of_sample_days_per_run = round(total_days_per_run * out_of_sample_size)

    calendar = pd.DataFrame()

    calendar['InSampleStarts'] = [start_date + timedelta(days=(out_of_sample_days_per_run * x))
                                  for x in range(runs)]
    calendar['InSampleEnds'] = [x + timedelta(days=in_sample_days_per_run)
                                for x in calendar['InSampleStarts']]

    calendar['OutSampleStarts'] = [start_date + timedelta(days=in_sample_days_per_run + n_gap) +
                                   timedelta(days=(out_of_sample_days_per_run * x))
                                   for x in range(runs)]
    calendar['OutSampleEnds'] = [x + timedelta(days=out_of_sample_days_per_run)
                                 for x in calendar['OutSampleStarts']]

    return calendar


calendar = build_rolling_calendar(start_date=scaled_df.index.min(), end_date=scaled_df.index.max(), out_of_sample_size=20, runs=10, n_gap=20)


# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

for i, row in calendar.iterrows():
    ax.plot([row['InSampleStarts'].year, row['InSampleEnds'].year], [i, i], color='blue')
    ax.plot([row['OutSampleStarts'].year, row['OutSampleEnds'].year], [i, i], color='red')

# Customize plot
ax.set_yticks(calendar.index.values[::-1])
ax.set_yticklabels(calendar.index.values[::-1])
ax.set_xlabel('Years')
ax.set_ylabel('Index')
ax.set_title('Sample Periods')
ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
from itertools import chain

X_train = []
X_test = []
y_train = []
y_test = []
for i, row in calendar.iterrows():
    train = scaled_df.loc[row['InSampleStarts']: row['InSampleEnds']].to_numpy()
    X_train.append(train[:, :-1])
    y_train.append(train[:, -1])
    test = scaled_df.loc[row['OutSampleStarts']: row['OutSampleEnds']].to_numpy()
    X_test.append(test[:, :-1])
    y_test.append(test[:,-1])
    if i == 0: 
        X_train_signals = scaled_df.loc[row['OutSampleStarts']:].to_numpy()


flattened_y_test = np.array(list(chain(*y_test)))

In [ ]:
svr_y_pred = []
svr = SVR(C=1, kernel='linear')
for i in range(len(X_train)):
    svr.fit(X_train[i], y_train[i])
    y_pred = svr.predict(X_test[i])
    svr_y_pred.extend(y_pred)
    #score = svr.score(y_pred, y_test[i])
    #print(f'batch {i} has a score of {score}')

# Assign the list of predictions to the DataFrame column
#sp500_d['Pred_log_return'].iloc[len((sp500_d)-200)] = np.concatenate(all_predictions)
#svr_score = svr.score(svr_y_pred,flattened_y_test)
#print(f'Accuracy of SVRegression is {svr_score}')


In [ ]:
scaled_df['predictions'] = np.zeros(scaled_df.shape[0])

# Determine the index range for appending predictions
start_index = scaled_df.shape[0] - 3934
end_index = scaled_df.shape[0]

# Assign predictions to the last 3934 indices of the DataFrame column
scaled_df['predictions'].iloc[start_index:end_index] = svr_y_pred

#scaled_df['predictions'] = svr_y_pred

In [ ]:
# Assuming df is your DataFrame containing the data
# Replace 'feature1' and 'feature2' with the names of the features you want to plot
feature1 = 'log_return'
feature2 = 'predictions'

# Create line plot
fig = go.Figure(data=go.Scatter(
    x=scaled_df.index,  # Assuming the index represents x-axis values
    y=scaled_df[feature1],
    mode='lines',
    name=feature1
))

# Add another line plot for the second feature
fig.add_trace(go.Scatter(
    x=scaled_df.index,  # Assuming the index represents x-axis values
    y=scaled_df[feature2],
    mode='lines',
    name=feature2
))

# Customize layout
fig.update_layout(
    title='Linear SVR - log_return Vs. Predictions on train dataset - WF 200,20',
    xaxis=dict(title='X Axis'),  # Customize x-axis label
    yaxis=dict(title='Y Axis'),  # Customize y-axis label
    hovermode='closest'
)

# Show plot
fig.show()
